In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
dfT = pd.read_csv('./data/thyroidDF.csv')
# retirando as colunas que não iremos utilizar para treinar o modelo
dfT.drop(columns=['patient_id', 'on_thyroxine', 'query_on_thyroxine', 'sick', 'I131_treatment',
                 'query_hypothyroid','pregnant','query_hyperthyroid','lithium', 'goitre',
                 'hypopituitary','psych','TSH_measured','T3_measured','TT4_measured','T4U_measured',
                  'FTI_measured','TBG_measured','referral_source'], inplace=True)

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(dfT.corr(), annot=True, cmap="magma");

In [ ]:
dfT.shape, dfT.columns

# indices que são numéricos
# 0 17 19 21 23 25 27

In [ ]:
dfT.head(15)

In [ ]:
dfT.describe()

In [ ]:
# Esse valor tem os dados desbalanceados devido uma quantidade de NaN presente
# E por termos valores vazios no dataset

dfT.mean()

In [ ]:
sns.histplot(x=dfT['TBG'], y=dfT['T4U'], kde=True, color='darkblue');

In [ ]:
sns.lmplot(data=dfT,
          x='TBG',
          y='T4U');

In [ ]:
sns.lmplot(data=dfT,
          x='TT4', y='FTI',
           hue='sex');

In [ ]:
sns.lmplot(data=dfT,
          x='TSH', y='T3');

In [ ]:
sns.lmplot(data=dfT,
          x='TSH', y='FTI');

In [ ]:
sns.lmplot(data=dfT,
          x='T3',
          y='TT4');

In [ ]:
sns.scatterplot(data=dfT, x='T4U', y='TT4', hue='sex')

In [ ]:
sns.scatterplot(data=dfT, x='T4U', y='TSH', hue='sex')

In [ ]:
sns.countplot(dfT['age'], palette='mako');
# dfT[dfT['age'] <= 0] >>>> sem idades negativas

In [ ]:
plt.hist(x = dfT['age'], color='lightgreen');

In [ ]:
sns.countplot(dfT['sex'], palette='mako');

In [ ]:
X = dfT.iloc[:, 0:11].values
X

In [ ]:
# Thyroxine-binding globulin (TBG), X[:,11] is one of three major transport proteins, which are primarily 
# responsible for binding to and transporting thyroid hormones to the necessary tissues
# dispersão do TBG pela idade

#f, ax = plt.subplots(figsize=(8, 8))
#sns.scatterplot(data=X, x=X[:, 0], y=X[:, 11], s=25, color='red');

In [ ]:
X[:,0].min(), X[:,0].max()

In [ ]:
y = dfT.iloc[:, 11].values
y

In [ ]:
## Label Enconder para tratamento de variáveis categóricas
from sklearn.preprocessing import LabelEncoder

In [ ]:
dfT_X = X

In [ ]:
dfT.columns

In [ ]:
dfT_X

In [ ]:
label_enconder_sex = LabelEncoder()
label_enconder_meds = LabelEncoder()
label_enconder_surgery = LabelEncoder()
label_enconder_tumor = LabelEncoder()
label_enconder_target = LabelEncoder()

In [ ]:
dfT_X

In [ ]:
dfT_X[:,1] = label_enconder_sex.fit_transform(dfT_X[:,1])
dfT_X[:,2] = label_enconder_meds.fit_transform(dfT_X[:,2])
dfT_X[:,3] = label_enconder_surgery.fit_transform(dfT_X[:,3])
dfT_X[:,4] = label_enconder_tumor.fit_transform(dfT_X[:,4])
dfT_X[:,11] = label_enconder_tumor.fit_transform(dfT_X[:,11])